In [ ]:
!pip install face_recognition face_recognition_models --no-dependencies

In [ ]:
from io import BytesIO
import time
import warnings

import numpy as np
from PIL import Image
from tqdm.notebook import tqdm

import ipywidgets

`ipywebrtc` provides an interface to media devices via browser. This is based on `ipywidgets` framework. We are going to use camera stream and image recorder functionality to read from web camera in this example.

In [ ]:
from ipywebrtc import CameraStream, ImageRecorder

In [ ]:
constraints = {
    "facing_mode": "user",
    "audio": False,
    "video": {
        "width": 640,
        "height": 480
    }
}
camera = CameraStream(constraints=constraints)

In [ ]:
recorder = ImageRecorder(stream=camera, recording=True)
name_input = ipywidgets.Text()

In [ ]:
def image_bytes_to_array(image_bytes):
    image = Image.open(BytesIO(image_bytes)).convert("RGB")
    return np.array(image)

In [ ]:
images = []
def on_image_changed(change):
    image = image_bytes_to_array(recorder.image.value)
    images.append((name_input.value, image))
    
recorder.image.observe(on_image_changed, "value")

`HBox` and `VBox` are simple layout classes provide by `ipywidgets` framework. To read more on layouts, refer to [this](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#Natural-sizes,-and-arrangements-using-HBox-and-VBox).

In [ ]:
ipywidgets.HBox([
    camera, 
    ipywidgets.VBox([
        ipywidgets.Label("Your Name Please!"), 
        name_input, 
        recorder
    ])
])

### Intentioal Exception

Raising an exception to break my restart and run all cells flow :P <br>
At this points, record multiple images in above widget with different subjects to collect training data.

In [ ]:
raise Exception()

### Old magic...

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### New magic !!

In [ ]:
from face_recognition import (
    face_locations as localize,
    face_encodings as encode
)

In [ ]:
subjects, encodings = [], []

iter_list = [x for x in enumerate(images)]
for i, (subject, image) in tqdm(iter_list):
    face_locations = localize(image, model="cnn")
    if len(face_locations) != 1:
        warnings.warn(f"ignoring image {i:2d} of {subject}... "
                      f"number of faces identified = {len(face_locations)}")
        continue
    
    face_encoding = encode(image, known_face_locations=face_locations)[0]
    encodings.append(face_encoding)
    subjects.append(subject)

encodings = np.stack(encodings)

print("encodings.shape =", encodings.shape)

In [ ]:
model = KNeighborsClassifier().fit(encodings, subjects)

In [ ]:
prediction_recorder = ImageRecorder(stream=camera, recording=True)
prediction_output = ipywidgets.Textarea()

In [ ]:
logs, changes, excepts = [], [], []

def on_snap(change):
    changes.append(change)
    try:
        image = image_bytes_to_array(change["new"])
        face_locations = localize(image, model="cnn")
        face_encodings = encode(image, known_face_locations=face_locations)
        logs.append(face_encodings)
        predictions = model.predict(face_encodings)
        prediction_output.value = "\n".join(predictions)
    except Exception as ex:
        prediction_output.value = ""
        excepts.append(ex)
    
prediction_recorder.image.observe(on_snap, "value")

In [ ]:
ipywidgets.HBox([
    camera,
    ipywidgets.VBox([
        prediction_recorder,
        prediction_output
    ])
])

In [ ]:
camera.close()